In [1]:
import ray
from ray import rllib, tune
import pandas as pd
import numpy as np
from datetime import datetime
import trading_gym
from trading_gym.registry.gaia.v9.env import GAIAPredictorsContinuousV9
from trading_gym.registry.gaia.v10.env import GAIAPredictorsContinuousV10
from trading_gym.ray.walkforward import WalkForwardRunner, WalkForwardResults
%matplotlib inline
print(trading_gym.__package__, trading_gym.__version__)
print(ray.__package__, ray.__version__)

trading-gym 0.8.1
ray 0.7.2


In [2]:
import ray
from ray import rllib, tune
from trading_gym.ray.logger import calculate_tearsheet, CustomLogger
from copy import deepcopy
# ray.init(num_cpus=8,ignore_reinit_error=True,object_store_memory= 10*100 )
ray.init(ignore_reinit_error=True)
#          object_store_memory = 50000000)

ray.__version__

2019-07-31 16:45:22,820	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-31_16-45-22_816547_86284/logs.
2019-07-31 16:45:23,009	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:16985 to respond...
2019-07-31 16:45:23,154	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:60375 to respond...
2019-07-31 16:45:23,161	INFO services.py:806 -- Starting Redis shard with 10.0 GB max memory.
2019-07-31 16:45:23,355	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-31_16-45-22_816547_86284/logs.
2019-07-31 16:45:23,358	WARNING services.py:1298 -- Warning: Capping object memory store to 20.0GB. To increase this further, specify `object_store_memory` when calling ray.init() or ray start.
2019-07-31 16:45:23,359	INFO services.py:1446 -- Starting the Plasma object store with 20.0 GB memory using /dev/shm.


'0.7.2'

In [3]:
env_config = dict()
env_config['folds'] =  {
    'training-set': [datetime.min, datetime(2008, 3, 18)],
    'test-set': [datetime(2008, 3, 19), datetime.max],
}
env = GAIAPredictorsContinuousV9(env_config)
env

In [4]:
## Create the config dict
config = ray.rllib.agents.ppo.DEFAULT_CONFIG.copy()
config['env'] = GAIAPredictorsContinuousV9
config['env_config'] = {
    'cost_of_commissions': tune.grid_search([0.00005]),
    'cost_of_spread': 0.0001,
}
config['gamma'] = 0.82  # 2 weeks

In [5]:
config['vf_clip_param'] = 0
config['vf_loss_coeff'] = 0
config['lambda'] = 0
config['use_gae'] = False

# need to have vf share layers if lstm is used
config['vf_share_layers'] = False 
config['use_lstm']: False
config['batch_mode'] = 'complete_episodes'
config['train_batch_size'] = tune.grid_search([4000])
config['sgd_minibatch_size'] = 128
config['num_sgd_iter'] = tune.grid_search([8])
config['entropy_coeff'] = tune.grid_search([1e-5])
config['kl_coeff'] = tune.grid_search([0.2])
config['kl_target'] = tune.grid_search([0.01])
config['clip_param'] = tune.grid_search([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1])

config['lr'] = tune.grid_search([1e-5])

# config['model']['custom_model'] = CNN.__name__


In [6]:
experiment = tune.Experiment(
    name='clip_exps',
    run=rllib.agents.ppo.PPOTrainer,
    stop={"timesteps_total": 100000},
    config=deepcopy(config),
#     This determines the number of times the grid search is run. s
    num_samples=1,
    local_dir='logs/clip_exps_aric',
    #checkpoint_freq=int(1e4 / config['train_batch_size']),  # checkpoint every 100k iters
    checkpoint_at_end=True,
    max_failures=0,
    loggers=[CustomLogger],
    
)

In [7]:
trials = tune.run_experiments(
    experiments=experiment,
    search_alg=tune.suggest.BasicVariantGenerator(),
    scheduler=tune.schedulers.FIFOScheduler(),
    verbose=1,
    reuse_actors=False,
    resume=False,)


2019-07-31 16:59:24,371	INFO tune.py:65 -- Did not find checkpoint file in logs/clip_exps_aric/clip_exps.
2019-07-31 16:59:24,379	INFO tune.py:233 -- Starting a new experiment.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/8 CPUs, 0/0 GPUs
Memory usage on this node: 38.5/67.5 GB



2019-07-31 16:59:24,792	WARNING signature.py:108 -- The function with_updates has a **kwargs argument, which is currently not supported.
2019-07-31 16:59:24,919	WARNING util.py:64 -- The `start_trial` operation took 0.14847111701965332 seconds to complete, which may be a performance bottleneck.
2019-07-31 16:59:24,990	WARNING signature.py:108 -- The function with_updates has a **kwargs argument, which is currently not supported.
2019-07-31 16:59:25,080	WARNING util.py:64 -- The `start_trial` operation took 0.10506844520568848 seconds to complete, which may be a performance bottleneck.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/8 CPUs, 0/0 GPUs
Memory usage on this node: 38.5/67.5 GB
Result logdir: logs/clip_exps_aric/clip_exps
Number of trials: 11 ({'RUNNING': 1, 'PENDING': 10})
PENDING trials:
 - PPO_GAIAPredictorsContinuousV9_1_clip_param=0.2,entropy_coeff=1e-05,cost_of_commissions=1e-05,kl_coeff=0.2,kl_target=0.01,lr=1e-05,num_sgd_iter=8,train_batch_size=4000:	PENDING
 - PPO_GAIAPredictorsContinuousV9_2_clip_param=0.3,entropy_coeff=1e-05,cost_of_commissions=1e-05,kl_coeff=0.2,kl_target=0.01,lr=1e-05,num_sgd_iter=8,train_batch_size=4000:	PENDING
 - PPO_GAIAPredictorsContinuousV9_3_clip_param=0.4,entropy_coeff=1e-05,cost_of_commissions=1e-05,kl_coeff=0.2,kl_target=0.01,lr=1e-05,num_sgd_iter=8,train_batch_size=4000:	PENDING
 - PPO_GAIAPredictorsContinuousV9_4_clip_param=0.5,entropy_coeff=1e-05,cost_of_commissions=1e-05,kl_coeff=0.2,kl_target=0.01,lr=1e-05,num_sgd_iter=8,train_batch_size=4000:	PENDING
 - PPO_GAIAPredictorsContinuousV9_5_clip

(pid=89161) 2019-07-31 17:00:09,022	INFO rollout_worker.py:301 -- Creating policy evaluation worker 1 on CPU (please ignore any CUDA init errors)
(pid=89161) 2019-07-31 17:00:09.123210: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=89158) 2019-07-31 17:00:09,339	INFO rollout_worker.py:301 -- Creating policy evaluation worker 1 on CPU (please ignore any CUDA init errors)
(pid=89158) 2019-07-31 17:00:09.399745: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=89161) 2019-07-31 17:00:09,601	INFO dynamic_tf_policy.py:313 -- Initializing loss function with dummy input:
(pid=89161) 
(pid=89161) { 'action_prob': <tf.Tensor 'default_policy/action_prob:0' shape=(?,) dtype=float32>,
(pid=89161)   'actions': <tf.Tensor 'default_policy/actions:0' shape=(?, 2) dtype=float32>,
(pid=89161)   'adva

...

2019-07-31 17:06:56,649	ERROR trial_runner.py:487 -- Error processing event.
Traceback (most recent call last):
  File "/home/Nicholas/.venv/lib/python3.6/site-packages/ray/tune/trial_runner.py", line 468, in _process_trial
    result, terminate=(decision == TrialScheduler.STOP))
  File "/home/Nicholas/.venv/lib/python3.6/site-packages/ray/tune/trial.py", line 533, in update_last_result
    self.result_logger.on_result(self.last_result)
  File "/home/Nicholas/.venv/lib/python3.6/site-packages/ray/tune/logger.py", line 239, in on_result
    _logger.on_result(result)
  File "/home/Nicholas/.venv/lib/python3.6/site-packages/trading_gym/ray/logger.py", line 69, in on_result
    for fold in result['trading-gym']:
KeyError: 'trading-gym'
2019-07-31 17:06:56,654	INFO ray_trial_executor.py:187 -- Destroying actor for trial PPO_GAIAPredictorsContinuousV9_10_clip_param=1.1,entropy_coeff=1e-05,cost_of_commissions=1e-05,kl_coeff=0.2,kl_target=0.01,lr=1e-05,num_sgd_iter=8,train_batch_size=4000. If 

(pid=101266) 2019-07-31 17:06:56,644	WARNING ppo.py:129 -- The magnitude of your environment rewards are more than infx the scale of `vf_clip_param`. This means that it will take more than inf iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.


2019-07-31 17:06:57,183	WARNING util.py:64 -- The `experiment_checkpoint` operation took 0.5248825550079346 seconds to complete, which may be a performance bottleneck.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/8 CPUs, 0/0 GPUs
Memory usage on this node: 39.2/67.5 GB
Result logdir: logs/clip_exps_aric/clip_exps
Number of trials: 11 ({'ERROR': 11})
ERROR trials:
 - PPO_GAIAPredictorsContinuousV9_0_clip_param=0.1,entropy_coeff=1e-05,cost_of_commissions=1e-05,kl_coeff=0.2,kl_target=0.01,lr=1e-05,num_sgd_iter=8,train_batch_size=4000:	ERROR, 1 failures: logs/clip_exps_aric/clip_exps/PPO_GAIAPredictorsContinuousV9_0_clip_param=0.1,entropy_coeff=1e-05,cost_of_commissions=1e-05,kl_coeff=0.2,kl_target=0.01,lr=1e-05_2019-07-31_16-59-24r06fnmw4/error_2019-07-31_17-00-59.txt, [3 CPUs, 0 GPUs], [pid=89162], 55 s, 1 iter, 4100 ts, 0.00508 rew
 - PPO_GAIAPredictorsContinuousV9_1_clip_param=0.2,entropy_coeff=1e-05,cost_of_commissions=1e-05,kl_coeff=0.2,kl_target=0.01,lr=1e-05,num_sgd_iter=8,train_batch_size=4000:	ERROR, 1 failures: logs/clip_exps_aric/clip_exps/PPO_GAIAPredictorsContinuousV9_1_clip_param=0.2,entropy_coeff=1e-05,cost_of_comm

TuneError: ('Trials did not complete', [PPO_GAIAPredictorsContinuousV9_0_clip_param=0.1,entropy_coeff=1e-05,cost_of_commissions=1e-05,kl_coeff=0.2,kl_target=0.01,lr=1e-05,num_sgd_iter=8,train_batch_size=4000, PPO_GAIAPredictorsContinuousV9_1_clip_param=0.2,entropy_coeff=1e-05,cost_of_commissions=1e-05,kl_coeff=0.2,kl_target=0.01,lr=1e-05,num_sgd_iter=8,train_batch_size=4000, PPO_GAIAPredictorsContinuousV9_2_clip_param=0.3,entropy_coeff=1e-05,cost_of_commissions=1e-05,kl_coeff=0.2,kl_target=0.01,lr=1e-05,num_sgd_iter=8,train_batch_size=4000, PPO_GAIAPredictorsContinuousV9_3_clip_param=0.4,entropy_coeff=1e-05,cost_of_commissions=1e-05,kl_coeff=0.2,kl_target=0.01,lr=1e-05,num_sgd_iter=8,train_batch_size=4000, PPO_GAIAPredictorsContinuousV9_4_clip_param=0.5,entropy_coeff=1e-05,cost_of_commissions=1e-05,kl_coeff=0.2,kl_target=0.01,lr=1e-05,num_sgd_iter=8,train_batch_size=4000, PPO_GAIAPredictorsContinuousV9_5_clip_param=0.6,entropy_coeff=1e-05,cost_of_commissions=1e-05,kl_coeff=0.2,kl_targe